## Face Recognition 

### Use Case

This project is about recognizing whether the person in the input image is exited in my Photo database. The idea comes from a Face Recognition task for employees' access by using their photo IDs to the company. Here are two tasks in my basic CNN project.

1. Require a user to input an image
2. Task1: Detecte whether the input image is about human faces
3. Task2: Finger whether the person in the input image is exited in my photo database
4. Task2 only is executed when task1 returns True

### Explanation

Human Face Detection: The HumanFaceDetector class uses a pre-trained model to detect human faces in the input image. The process_face_recognition function checks if the image contains a human face and returns a message accordingly.

Image Similarity Check: The ImageSimilarityCNN class reads all images in a local folder, converts them into vectors, and performs similarity analysis with the input image.

Main Function: The main function first checks if the input image contains a human face. If it does, it proceeds to perform the similarity analysis. If not, it prints "It is not a human face image" and exits.

In [12]:
import os
import cv2
import torch
import numpy as np
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import img_to_array
from transformers import AutoFeatureExtractor, AutoModelForObjectDetection

In [14]:
# First Task: Human Face Detection
class HumanFaceDetector:
    def __init__(self):
        model_name = "facebook/detr-resnet-50"
        self.feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)
        self.model = AutoModelForObjectDetection.from_pretrained(model_name)
    
    def detect_human_face(self, image_path):
        if not os.path.exists(image_path):
            print(f"Error: Image file {image_path} does not exist.")
            return False, None

        img = cv2.imread(image_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        inputs = self.feature_extractor(images=img_rgb, return_tensors="pt")
        
        outputs = self.model(**inputs)
        
        target_sizes = torch.tensor([img.shape[:2]])
        results = self.feature_extractor.post_process_object_detection(outputs, target_sizes=target_sizes)[0]
        
        for score, label in zip(results["scores"], results["labels"]):
            if (self.model.config.id2label[label.item()] == 'person' and score > 0.7):
                return True, img
        
        return False, None
    
def process_face_recognition(image_path):
    face_detector = HumanFaceDetector()
    is_human, detected_image = face_detector.detect_human_face(image_path)
    
    if not is_human:
        return "It is not a human face image"
    
    return "Human face detected!"


In [65]:
# Second Task: Image Similarity Check
class ImageSimilarityCNN:
    def __init__(self, image_directory):
        self.image_directory = image_directory
        self.model = self.build_model()
        self.image_vectors = self.load_images()

    def build_model(self):
        model = Sequential([
            Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
            MaxPooling2D((2, 2)),
            Conv2D(64, (3, 3), activation='relu'),
            MaxPooling2D((2, 2)),
            Flatten(),
            Dense(128, activation='relu'),
            Dense(128, activation='relu')
        ])
        return model

    def preprocess_image(self, image_path):
        image = Image.open(image_path).convert('RGB')  # Convert image to RGB
        image = image.resize((224, 224))
        image_array = img_to_array(image)
        image_array = np.expand_dims(image_array, axis=0)
        return image_array

    def load_images(self):
        image_vectors = []
        for filename in os.listdir(self.image_directory):
            if filename.endswith(".jpg") or filename.endswith(".png"):
                image_path = os.path.join(self.image_directory, filename)
                image_array = self.preprocess_image(image_path)
                image_vector = self.model.predict(image_array)
                image_vectors.append(image_vector.flatten())
        return image_vectors

    def is_similar(self, input_image_path):
        input_image_array = self.preprocess_image(input_image_path)
        input_image_vector = self.model.predict(input_image_array).flatten()
        for image_vector in self.image_vectors:
            similarity = np.dot(input_image_vector, image_vector) / (np.linalg.norm(input_image_vector) * np.linalg.norm(image_vector))
            if similarity > 0.7:  # Threshold for similarity
                return True
        return False

Here, I also try to compare the performances of ResNet50 and InceptionV3 models with optimizations to prevent overfitting issues. I found somethings very interesting. 

1. InceptionV3 computed faster to get the output as it is a parallel computing architecture framework.
2. ResNet50 seems to get more details to set a lower similarity value than other models since it can learn very deep layers by skipping one hidden layer in each step.  


Explanation:

Data Augmentation: The ImageDataGenerator is used to apply random transformations to the images, such as rotation, width/height shift, shear, zoom, and horizontal flip. This helps in generating variations of the training images to prevent overfitting.
Loading Images with Augmentation: The load_images method now uses the ImageDataGenerator to augment each image before converting it to a vector.


### ResNet50

In [34]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array

In [77]:
# Second Task: Image Similarity Check with ResNet-50 and Data Augmentation
class ImageSimilarityResNet50:
    def __init__(self, image_directory):
        self.image_directory = image_directory
        self.model = self.build_model()
        self.image_vectors = self.load_images()

    def build_model(self):
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dropout(0.5)(x)  # Add dropout to prevent overfitting
        x = Dense(128, activation='relu')(x)
        model = Model(inputs=base_model.input, outputs=x)
        return model

    def preprocess_image(self, image_path):
        image = Image.open(image_path).convert('RGB')  # Convert image to RGB
        image = image.resize((224, 224))
        image_array = img_to_array(image)
        image_array = np.expand_dims(image_array, axis=0)
        return image_array

    def load_images(self):
        image_vectors = []
        datagen = ImageDataGenerator(
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest'
        )
        for filename in os.listdir(self.image_directory):
            if filename.endswith(".jpg") or filename.endswith(".png"):
                image_path = os.path.join(self.image_directory, filename)
                image_array = self.preprocess_image(image_path)
                for batch in datagen.flow(image_array, batch_size=1):
                    image_vector = self.model.predict(batch)
                    image_vectors.append(image_vector.flatten())
                    break  # We only need one augmented image per original image
        return image_vectors

    def is_similar(self, input_image_path):
        input_image_array = self.preprocess_image(input_image_path)
        input_image_vector = self.model.predict(input_image_array).flatten()
        for image_vector in self.image_vectors:
            similarity = np.dot(input_image_vector, image_vector) / (np.linalg.norm(input_image_vector) * np.linalg.norm(image_vector))
            if similarity > 0.5:  # Threshold for similarity
                return True
        return False


### InceptionV3

In [45]:
from tensorflow.keras.applications import InceptionV3

In [69]:
# Second Task: Image Similarity Check with InceptionV3 and Data Augmentation
class ImageSimilarityInceptionV3:
    def __init__(self, image_directory):
        self.image_directory = image_directory
        self.model = self.build_model()
        self.image_vectors = self.load_images()

    def build_model(self):
        base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dropout(0.5)(x)  # Add dropout to prevent overfitting
        x = Dense(128, activation='relu')(x)
        model = Model(inputs=base_model.input, outputs=x)
        return model

    def preprocess_image(self, image_path):
        image = Image.open(image_path).convert('RGB')  # Convert image to RGB
        image = image.resize((224, 224))
        image_array = img_to_array(image)
        image_array = np.expand_dims(image_array, axis=0)
        return image_array

    def load_images(self):
        image_vectors = []
        datagen = ImageDataGenerator(
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest'
        )
        for filename in os.listdir(self.image_directory):
            if filename.endswith(".jpg") or filename.endswith(".png"):
                image_path = os.path.join(self.image_directory, filename)
                image_array = self.preprocess_image(image_path)
                for batch in datagen.flow(image_array, batch_size=1):
                    image_vector = self.model.predict(batch)
                    image_vectors.append(image_vector.flatten())
                    break  # We only need one augmented image per original image
        return image_vectors

    def is_similar(self, input_image_path):
        input_image_array = self.preprocess_image(input_image_path)
        input_image_vector = self.model.predict(input_image_array).flatten()
        for image_vector in self.image_vectors:
            similarity = np.dot(input_image_vector, image_vector) / (np.linalg.norm(input_image_vector) * np.linalg.norm(image_vector))
            if similarity > 0.7:  # Threshold for similarity
                return True
        return False


In [79]:
# Main Function
def main():
    input_image_path = input("Enter the path to the input image: ")
    
    # First Task: Check if the image contains a human face
    face_recognition_result = process_face_recognition(input_image_path)
    if face_recognition_result == "It is not a human face image":
        print(face_recognition_result)
        return
    
    # Second Task: Perform similarity analysis if the image contains a human face
    image_directory = 'people_images'

    #model1:CNN
    print("CNN model")
    cnn1 = ImageSimilarityCNN(image_directory)
    if cnn1.is_similar(input_image_path):
        print("true!")
    else:
        print("false!")    
        
    #model2: ResNet50
    print("ResNet50 model")
    cnn2 = ImageSimilarityResNet50(image_directory)
    if cnn2.is_similar(input_image_path):
        print("true!")
    else:
        print("false!")  
        
    #model3: InceptionV3
    print("InceptionV3 model")
    cnn3 = ImageSimilarityInceptionV3(image_directory)
    if cnn3.is_similar(input_image_path):
        print("true!")
    else:
        print("false!")      


if __name__ == "__main__":
    main()

Enter the path to the input image:  test_images/test6.png


Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


CNN model
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
true!
ResNet50 model
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
true!
InceptionV3 model
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/s